In [1]:
import os, sys
import threading, logging, time
import multiprocessing
import gzip
import datetime
from datetime import timedelta
import time
from kafka import KafkaConsumer, KafkaProducer
import json

In [ ]:
if __name__ == "__main__":
    consumer = KafkaConsumer(bootstrap_servers='backend-0-3:6667',
                             auto_offset_reset='earliest',
                             consumer_timeout_ms=30000) #timeout ms has to be adjusted so that it does not break for loop many times.
                             #value_deserializer=lambda m: json.loads(m.decode('utf-8')))
         
    consumer.subscribe(['atlanta_sprint_twitter'])
    #Initiate counters for number of records written to file and number of errors
    fcntr = 0
    ecntr = 0
    #Run this piece of code in loop by specifying while True condition
    while True:
        #Initiating an empty set to store tweet IDs
        tweet_ids = set()   
        #Current time
        t1 = datetime.datetime.strftime(datetime.datetime.now(),"%Y%m%d%H")
        #Create temporary JSON file to write data locally. This file will be pushed to kafka every hour.
        f = open('twdata.json', 'wb')
        #Iterate through each message received through kafka consumer
        for message in consumer:
            #the data is deserialized and message is a tuple with 7th element containing the json data. 
            #JSON data is loaded using json.loads.
            tweet_data = json.loads(message.value.decode("utf-8"))
            #using a try except block to make sure that the errors do not stop the code and the errors are counted. 
            try:
                #Checking if the tweet ID is already present in the list of unique tweet IDs
                if tweet_data['id'] not in tweet_ids:
                    #if it is a unique tweet, write it to the file.
                    f.write(message.value)
                    fcntr+=1
                    #add the unique tweet ID to the set.
                    tweet_ids.add(tweet_data['id'])
                
                #If the current time is greater than the time noted at the beginnig of the hour (change of hour), 
                #then push file to HDFS, reset the tweet IDs set and reopen the temporary file.
                if datetime.datetime.strftime(datetime.datetime.now(),"%Y%m%d%H") > t1:
                    # If time exceeds one hour then push the file to HDFS cluster
                    f.close()
                    fname = '/data/atl_sprint_2018/twitter_archive/tw%s.json' % (datetime.datetime.strftime(datetime.datetime.now(), "%Y%m%dT%H%M"))
                    var = os.system('hdfs dfs -put twdata.json %s' %(fname))
                    tweet_ids = set()   #Initiating an empty list
                    #Reassign the current time in YYYYMMDDH to t1. 
                    t1 = datetime.datetime.strftime(datetime.datetime.now(),"%Y%m%d%H")
                    #Reopen the temp file in write binary mode
                    f = open('twdata.json', 'wb')
                    print("%s files written at %s excluding %s errors" %(fcntr,t1,ecntr))
                    ecntr=fcntr=0
                    break
            except:
                ecntr+=1
        print('end of while loop')

we are here again!
